# <span style="color:cornflowerblue"> Gerald Jones, Kimon Swanson, Alex Hines</span>
# <span style="color:cornflowerblue"> Home Work 5: Farmers Problem</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links:
1. [Data Display section](#Data-Display)
2. [Method Definitions](#Method-Definitions)
3. [Model Formulation Problem 1](#Model-Formulation-1)
4. [Gurobi Implementation Problem 1](#prob1)
5. [Solution Discussion Problem 1](#solution1)
6. [Model Formulation Problem 2](#Model-Formulation-2)
7. [Gurobi Implementation Problem 2](#prob2)
8. [Solution Discussion Problem 2](#solution2)
9. [Model Formulation Problem 3](#Model-Formulation-3)
10. [Gurobi Implementation Problem 3](#prob3)
11. [Solution Discussion Problem 3](#solution3)

## Problem Description:
> The farmer's problem Consider and farmer in east Tennessee who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop. The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler. Any production in excess of the feeding requirement would be sold. Over the last decade, mean selling prices have been $\text{\$170}$ and $\text{\$150}$ per ton of wheat and corn, respectively. The purchase prices are 40% more than this due to the wholesaler's margin and transportation costs.
<br><p>
> Another profitable crop is **sugar beet**, which he expects to sell at $\text{\$36}$/T; however, the EU imposes a quota on sugar beet production. Any amount in excess of the quota can be sold onlv at $\text{\$10}$/T. The farmer'squota for next year is 6,000 T. Based on past experience, the farmer knows that he mean yield on his land is roughly 2.5 T,3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. The table on the next slide summarizes these data and provides the planting costs for these crops. Formulate a linear program to maximize total profit (planting cost —revenue from sales).</p>

# Extentions To Accomplish:
1. [Suppose the farmer allocates 120 acres to wheat, 80 to corn, and 300 for sugar beets. But also suppose that the 
   yields are not certain but are subject to the same 3 scenarios we discussed in class (above average, average, below 
   average, each with equation probability. What is the expected annual profit?](#prob1)
    - [Problem 1 Method Defintion and Execution Cell](#prob1-meth)
2. [Consider an extension of the problem in which prices also are uncertain. Specifically, when yields are above 
   average prices for corn and wheat go down by 10% and when yields are below average, they go up by 10%.
   Assume sugar beet prices are not affected by yields. Formulate and solve a stochastic linear program and solve 
   it with a solver.](#prob2)
3. Consider a different version of the problem in which the farmer possesses four fields of sizes 185, 145, 105, and 
   65, respectively. The total of 500 acres is unchanged. However, the farmer wishes to only plant one type of crop 
   on each field. Formulate and solve a stochastic program and solve it with a solver

## Notes and Observations
* Total of 500 acres

* 3 possible crops:
    - wheat 
        + (>= 200 T through crops or purchase)
        + $\text{\$170}$/T selling price
        + $\text{\$170}$*40  + $\text{\$170}$ purchase price = $\text{\$238}$/T
        + AVG yield: 2.5 T
    - corn  
        + (>= 240 T through crops or purchase)
        + $\text{\$150}$/T selling price
        + $\text{\$150}$*40 + $\text{\$150}$ purchase price = $\text{\$210}$/T
        + AVG yield: 3 T
    - sugar beats
        + selling price is tiered
            - below quota:= $\text{\$36}$/T
            - above quota:= $\text{\$10}$/T
        + quota:= 6000 T
        + AVG yield: 20 T

* any purchase of wheat or corn over the required amount is sold for profit at indicated selling price

## Assumptions:


# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [2]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *
import numpy as np

# name for the conversion to pdf method from _NOTE_BOOK_UTILS
notebook_title = "_HW4_Problem1.ipynb"

<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display & Python Variable Assignments</center></span></a>

## Below cell defines several variables representing the data and constraints 

In [29]:
# display data for problem
# data is represented by the instantiated dataframe show here and displayed in the output below

# data for the problem
data_df = pd.DataFrame(
{
    "Wheat":[2.5, 150, 170, 170, 238, 200], 
    "Corn": [3,   230, 150, 150, 210, 240],
    "Sugar Beets":[20, 260, 36, 10, 0, 0],
},
    index=["Yield", "Planting Cost", "Selling Cost1", "Selling Cost2", "Price", "min"]
)

display(data_df)

# used to calculate the above/below average yields
average_rate = .2 # (percentage above/below the respective high and low average scenerios)

# price percentage difference in the low and high average yield scenerios 
price_rate = .1



#######################################################################################
# Initial amount of acreage devoted to each crop type
crop_acreage = [
    120, # acres of wheat to plant
    80,  # acres of corn to plant
    300  # acres of sugar beets to plant
]

print("Initial Allotted Acreage:")
print("Wheat: {}".format(crop_acreage[0]))
print("Corn: {}".format(crop_acreage[1]))
print("Sugar Beets: {}".format(crop_acreage[2]))

# adjustment to yield rate based on scenerio
# rows-> crops, columns->scenerio
# In this problem the scenerios are expected to see a 20% decrease, or increase 
# for the below and above average cases respectively
crop_yieldRate_scenerio = [-.2, 0, .2]  # below average, average, above average-->wheat
               

crop_price_scenerio =[-.1, 0, .1]  # below average, average, above average-->wheat


feedCrops = ["Wheat","Corn"]
# Remove this it is not needed the information is in the DF
# Minimum amount of wheat/Corn that must be yielded for feed crop purposes
# minT_dict = {
#     "Wheat":200,        # minimum tons of wheat needed for feed
#     "Corn":240,         # minimum tons of corn needed for feed
# }

# 
print("\t\t\t------------------------- Minimum crop resources for feed crops")
for c in feedCrops:
    print("Require a minimum of {} tons of {}".format(data_df.loc["min",c], c))
print("-------------------------------------------------")
# print(ysc)
# print(len(ysc))
quota = 6000               # yield level that when exceeded causes price of sugar beets-->reduced/penalized

# Probabilities for he below, average, and above average cases
scenerio_probs = [
    1/3,  # below average yield probability-->
    1/3,  # average yield probability
    1/3,  # above average yield probability
]


,Wheat,Corn,Sugar Beets
Yield,2.5,3,20
Planting Cost,150.0,230,260
Selling Cost1,170.0,150,36
Selling Cost2,170.0,150,10
Price,238.0,210,0
min,200.0,240,0


Initial Allotted Acreage:
Wheat: 120
Corn: 80
Sugar Beets: 300
			------------------------- Minimum crop resources for feed crops
Require a minimum of 200.0 tons of Wheat
Require a minimum of 240 tons of Corn
-------------------------------------------------


# <a id=Method-definitions><span style="color:red"><center>Method Definitions</center></span></a>

# <a id=prob1-meth>Problem 1 Solver method</a>
[Problem 1 Method Defintion and Execution Cell](#prob1-meth) | [Problem 1 Solution](#prob1)

In [77]:
# calculate annual profit with yield uncertainty
def objective_uncertain_yields(df, xw, ysc, quota, minT_dict, scenerio_probs=None, 
                               equally_likely=True, ):
    """
            df: dataframe containing columns Wheat, Corn, Sugar Beets, and indices Planting Cost, Yield, Selling Cost1, 
                and Selling Cost2. The Yield,"Crop" is the expected yield rate of the given crop amount xw
            xw: list representing the amount of each crop that was planted. takes the form:
                    xw[0] --> amount of Wheat planted
                    xw[1] --> amount of Corn planted  
                    xw[2] --> amount of Sugar Beets planted  
            ysc: list of lists for the expected % of planted crop yielded. Takes the form:
                    ysc[scenerio, crop]
                    crop:  {0 (wheat), 1 (corn), 2 (Sugar Beets)}
                    scenerio: {0 (below average percentage of planted yielded), 
                               1 (just 1 representing the expected average yield), 
                               2 (above average percentage of planted yielded)} 
    """
    #              ***** calculate and return annual costs under uncertainty *****
    
    
    if scenerio_probs is None:
        scenerio_probs = list()
        num_cases = len(ysc.shape[1])
        for i in range(num_cases):
            scenerio_probs.append(1.0/num_cases)
    
    # amount of each crop planted
    wheat_planted = xw[0]
    corn_planted = xw[1]
    sugarbeets_planted = xw[2] 
    
    # average expected yields
    avg_wheat = df.loc["Yield", "Wheat"]
    avg_corn = df.loc["Yield", "Corn"]
    avg_sugarbeets = df.loc["Yield", "Sugar Beets"]
    
    # costs for planting the given acreages of each type of crop
    wheat_costs = df.loc["Planting Cost", "Wheat"]*wheat_planted
    corn_costs = df.loc["Planting Cost", "Corn"]*corn_planted
    sugarbeets_costs = df.loc["Planting Cost", "Sugar Beets"]*sugarbeets_planted
    
    # Total planting costs
    planting_costs = wheat_costs + corn_costs + sugarbeets_costs
    print("\t\t\t---------------Planting Costs for Given Crop Allotments---------------")
    print("\tWheat planting costs: {:>22.0f}".format(wheat_costs))
    print("\tCorn planting costs: {:>23.0f}".format(corn_costs))
    print("\tSugar beets planting costs: {:>16.0f}".format(sugarbeets_costs))
    print("\tTotal Planting Costs: {:>23.0f}".format(planting_costs))
    print("------------------------------------------------------------------------\n")
    
    # yield amounts for each scenerio
    # (below average= average + average*-below_average_rate = average(1 - rate))
    # (average = average + average*0 = average*(1 + 0)
    # (above average= average + average*above_average_rate = average(1 + rate))-->
    # make list for each crop where [below_average_yield, average_yield, above_average_yield]
    wheat_yields = [avg_wheat*(1+ysc[0, scenerio])*xw[0] for scenerio in range(len(ysc))]
    corn_yields = [avg_corn*(1+ysc[1, scenerio])*xw[1] for scenerio in range(len(ysc))]
    sugarbeets_yields = [avg_sugarbeets*(1+ysc[2, scenerio])*xw[2] for scenerio in range(len(ysc))]
    print("\t\t\t---------------Expected Yields (tons) for Each Possible Case---------------")
    print("\nWheat yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(wheat_yields[0], 
                                                                                        wheat_yields[1],
                                                                                        wheat_yields[2]))
    
    print("\nCorn yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(corn_yields[0], 
                                                                                     corn_yields[1],
                                                                                     corn_yields[2]))
    print("\nSugarbeet yields:\n\tBelow Average {:.2f},\n\tAverage {:.2f},\n\tAbove Average {:.2f}".format(sugarbeets_yields[0], 
                                                                                             sugarbeets_yields[1],
                                                                                             sugarbeets_yields[2]))
    print("------------------------------------------------------------------------\n")
#     wheat_profits = df.loc["Yield", "Wheat"]*((1-ysc[0, 0]) + ysc[1, 0] + (1+ysc[2, 0]))*xw[0]*df.loc["Selling Cost1", "Wheat"]
#     corn_profits = df.loc["Yield", "Corn"]*((1-ysc[0, 1]) + ysc[1, 1] + (1+ysc[2, 1]))*xw[1]*df.loc["Selling Cost1", "Corn"]
    
    # profits based on different scenerios
    # excess_wheat = a if a < b else b
    # use yields to: 
    #         1) calculate the amount of wheat/corn that needs to be purchased
    #         2) calculate any excess wheat/corn that can be sold for profit
    
    #                 1) Amount to purchase for each scenerio
    excess_wheat, missing_wheat = list(), list()
    excess_corn, missing_corn = list(), list()
    excess_sugarbeets, justright_sugarbeets = list(), list() 
    
    
    for wheatSi, cornSi, sugarbeetSi in zip(wheat_yields, corn_yields, sugarbeets_yields):
        # calculate the excess for scenerio i(below, average, above)
        excess_wheat.append(wheatSi - minT_dict["Wheat"] if wheatSi - minT_dict["Wheat"] >= 0 else 0 )
        missing_wheat.append(minT_dict["Wheat"] - wheatSi if minT_dict["Wheat"] - wheatSi >= 0 else 0)
        
        # 
        excess_corn.append(cornSi - minT_dict["Corn"] if cornSi - minT_dict["Corn"] >= 0 else 0 )
        missing_corn.append(minT_dict["Corn"] - cornSi if minT_dict["Corn"] - cornSi >= 0 else 0)
    
        # 
        excess_sugarbeets.append(sugarbeetSi - quota  if sugarbeetSi > quota else 0)
        justright_sugarbeets.append(quota  if sugarbeetSi > quota else sugarbeetSi)
    
    
    # Profits:
    wheat_profits = (np.array(excess_wheat) * df.loc["Selling Cost1", "Wheat"]).flatten()
    corn_profits = (np.array(excess_corn) *  df.loc["Selling Cost1", "Corn"]).flatten()
    sugarbeets_profits = np.array(excess_sugarbeets) * df.loc["Selling Cost2", "Sugar Beets"] + np.array(justright_sugarbeets) * df.loc["Selling Cost1", "Sugar Beets"]
    
    # Purchase Costs:
    print("\t\t\t---------------Expected Excess Crops for each case---------------")
    wm_fl = np.array(missing_wheat).flatten()
    we_fl = np.array(excess_wheat).flatten()
    cm_fl = np.array(missing_corn).flatten()
    ce_fl = np.array(excess_corn).flatten()
    
    print("Excess Wheat:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                we_fl[0], 
                                                                                we_fl[1], 
                                                                                we_fl[2]))
    print("Excess Corn:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                ce_fl[0], 
                                                                                ce_fl[1], 
                                                                                ce_fl[2]))
    
    print("------------------------------------------------------------------------\n")
    
    print("\t\t\t---------------Expected Crops That need to be purchased for each case---------------")
    print("\nWheat missing:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                wm_fl[0], 
                                                                                wm_fl[1], 
                                                                                wm_fl[2]))
    print("\nCorn missing:\n\tBelow Average{:>23.0f},\n\tAverage{:>23.0f},\n\tAbove Average{:>23.0f}".format(
                                                                                cm_fl[0], 
                                                                                cm_fl[1], 
                                                                                cm_fl[2]))
    

    
    # calculate the cost of purchasing needed wheat and corn
    wheat_purhased_costs = np.array(missing_wheat) * df.loc["Price", "Wheat"]
    corn_purhased_costs = np.array(missing_corn).flatten() * df.loc["Price", "Corn"]
    print("\t\t\t---------------Purchase Costs---------------")
    print("wheat purchased: ", wheat_purhased_costs)
    print("Corn purchased: ", corn_purhased_costs)
    print("------------------------------------------------------------------------\n")
    
    
    
    
    # Total purchasing costs
    purchasing_costs = wheat_purhased_costs + corn_purhased_costs
    
    print()
    print("\t\t\t---------------Sales profits---------------")
    print("wheat profits: ", wheat_profits)
    print("corn profits: ", np.array(corn_profits).flatten())
    # sugar beets to sell at the different levels of sales cost
    print("Sugar beets within quota: ", justright_sugarbeets)
    print("Sugar beets above quota: ", excess_sugarbeets)
    print("Sugar beets profits: ", sugarbeets_profits)
    # total profits from sales
    sale_profits = wheat_profits + corn_profits + sugarbeets_profits
    print("Sales Profits Total: ", sale_profits)
    print("------------------------------------------------------------------------\n")
    
    print("\t\t\t---------------Total Costs and Profits---------------")
    print("Sales Profits Total: ", sale_profits)
    print("Planting Costs: ", planting_costs)
    print("Purchasing Costs: ", purchasing_costs)
    print("------------------------------------------------------------------------\n")
    
    b_avg_obj = sale_profits[0] - planting_costs - purchasing_costs[0]
    avg_obj = sale_profits[1] - planting_costs - purchasing_costs[1]
    a_avg_obj = sale_profits[2] - planting_costs - purchasing_costs[2]
    print()
    print("\t\t\t---------------Expected Profits for Each Yield Scenerio---------------")
    print("Below Average Expected Profits: {}".format(b_avg_obj))
    print("Average Expected Profits: {}".format(avg_obj))
    print("Above Average Expected Profits: {}".format(a_avg_obj))
    print("------------------------------------------------------------------------\n")
    
    expected_profit = scenerio_probs[0]*below_average_case +\
                        scenerio_probs[1]*average_case +\
                        scenerio_probs[2]*above_average_case
    
    return (b_avg_obj, avg_obj, a_avg_obj),  expected_profit 


# # call main method that performs calculation
# three_cases, expected_profit =  objective_uncertain_yields(
#                                                             data_df, #(Yield, Price, Selling-Prices)/Crop
#                                                             crop_acreage,    # amount of aceres devoted to each crop
#                                                             crop_yieldRate_scenerio,   # yield rate adjustment/yield scenerio
#                                                             quota, # sugar beets yield quota
#                                                             minT_dict,  # key:=crop, val=minimum tons of crop for feed
#                                                             scenerio_probs, # probability of each scenerio
#                                                             equally_likely=True, 
#                                                           )

# # Parse below, average, and above average expected profits
# below_average_case = three_cases[0]
# average_case = three_cases[1]
# above_average_case = three_cases[2]

# # Print the calculated Expected cost based on the three cases for yield amounts
# # and the given crop acreage allotments
# print("Expected Profit: {}".format(expected_profit))


# expected_profit = probs[0]*below_average_case + probs[1]*average_case + probs[2]*above_average_case
# print("Above average case Profit: {}".format(above_average_case))
# print("Average case Profit: {}".format(average_case))
# print("Below average case Profit: {}".format(below_average_case))

# # calculate it again just to do a sanity check
# print("Expected Profit: {}".format(expected_profit))

Initial Allotted Acreage:
Wheat: 120
Corn: 80
Sugar Beets: 300
			------------------------- Minimum crop resources for feed crops
Require a minimum of 200 tons of Wheat
Require a minimum of 240 tons of Corn
-------------------------------------------------
			---------------Planting Costs for Given Crop Allotments---------------
	Wheat planting costs:                  18000
	Corn planting costs:                   18400
	Sugar beets planting costs:            78000
	Total Planting Costs:                  114400
------------------------------------------------------------------------

			---------------Expected Yields (tons) for Each Possible Case---------------

Wheat yields:
	Below Average 240.00,
	Average 300.00,
	Above Average 360.00

Corn yields:
	Below Average 192.00,
	Average 240.00,
	Above Average 288.00

Sugarbeet yields:
	Below Average 4800.00,
	Average 6000.00,
	Above Average 7200.00
------------------------------------------------------------------------

			---------------Ex

# <a id=prob2>Problem 2</a>:

[]() | []()
> Consider an extension of the problem in which prices also are uncertain. Specifically, when **yields are above 
average, prices for corn and wheat go down by 10%** and when **yields are below average, they go up by 10%**.
**Assume sugar beet prices are not affected by yields**. Formulate and solve a stochastic linear program and solve 
it with a solver.

## Notes:
> In this version there are levels of crop sales prices in a similar manner to the levels of sales prices for sugar beets in the previous problems. Here, the sales price of a feed crop is based on the expected yield as shown below: 
* When (feed crop yields > average), feed_crop_yields=average(1+.2): # market surplus
    - prices go down 10%
        + prices * .9
* When (feed crop yields = average), feed_crop_yields=average
    - prices are expected average
        + prices * 1
* When (feed crop yields < average), feed_crop_yields=average(1-.2): # market shortages
    - prices go up 10%
        + prices * 1.1
        
* Each case is equally likely
* Have to account for the three possibilities 

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 2</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-2)
* [Variables](#Variables-2)
* [Equations and Constraints](#Equations-and-Constraints-2)
* [Objective](#Objective-2)

## <a id=Parameters-and-Sets-2><span style="color:DarkBlue">Parameters and Sets 2:</span></a>

### $\textbf{C}  \quad \quad \quad \text{set of possible crops {wheat(1), corn(2), sugar-beets $\leq$ 6KT(3), sugar-beets > 6KT (4)} } c \in \textbf{C}$ 
### $S \quad \quad \quad \text{ expected level for yield {below average(1), average(2), below average(3)} where s} \in S$
### $\mu_s \quad \quad \quad \text{ probability of scenerio s occuring}$
### $X_c \quad \quad \quad \text{acres allocated to crop c, }\in\{wheat(1), corn(2), sugarbeets(3)\}$
### $Y_{s,c} \quad \quad \quad \text{ tons purchased in scenerio s for crop c $\in$ \{1, 2\}}$
### $E_{s,c} \quad \quad \quad \text{ Expected yield (tons) in scenerio s for crop c}$
### $W_{s,c} \quad \quad \quad \text{ tons of crop c sold in scenerio s}$
### $P_{c} \quad \quad \quad \text{ purchase cost for crop } c \in \{wheat, corn\}$
### $L_{c} \quad \quad \quad \text{ planting cost ($\$$/acre) for crop } c \in C$
### $C_{s,c} \quad \quad \quad \text{ purchase price for crop c in scenerion s, c }\in \{wheat(1), corn(2)\}$
### $F_{s,c} \quad \quad \quad \text{ sale price for crop c}$
### $M_{c} \quad \quad \quad \text{ minimum amount of feed crop c} \in {1, 2}$
### $A \quad \quad \quad\text{ maximum amount of acreage available}$
### $r_s \quad \quad \quad \text{ the purchase price for yield scenerio s}$
### $\rho_{s,c} \quad \quad \quad \text{ average expected ton/acre yield rate for crop c for scenerio s}$
### $\mathbb{G} \quad \quad \quad \quad \text{Total Planting costs}$
### $\mathbb{R}_s \quad \quad \quad \quad \text{Total Purchasing costs for feed for scenerio s}$
### $\mathbb{H}_s \quad \quad \quad \quad \text{Total sales profits from all excess crops for scenerio s}$
### $\mathbb{O}_s \quad \quad \quad \quad \text{objective value for scenerio s}$


## <a id=Equations-and-Constraints-2><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# First Stage: How much to plant
> ### Nonzero acreage constraint: $X_c \geq 0, \text{ }\forall c$
> ### Maximum Acreage Available: $\sum_{c}^{|C|}(X_c) \leq A$
> ### Planting Costs: $\mathbb{G} = \sum_{c=1}^{3}(X_c \cdot L_c)$


# Second Stage: Expected Yields & Costs and Sales
### Below Average Yield:
> ### Expected yield below average expected: $E_{1, c} = \rho_{c}(.8), \forall c$
> ### purchase cost constraint based on yield: $C_{1,c} == P_c \cdot (1.1) \implies E_{1,c} == \rho_{c} \cdot (.8), \text{  c} \in \{0,1\}$
> ### Total Sale Profits costs: $\mathbb{H}_1 = \sum_{c=1}^{4}(W_{1,c} \cdot F_{1,c} \cdot (1.1))$
> ### Total Purchase costs: $\mathbb{R}_1 = \sum_{c=1}^{2}(Y_{1,c} \cdot C_{1,c})$
### Average:
> ### Expected yield below average expected: $E_{2, c} = \rho_{c}, \forall c$
> ### purchase cost constraint based on yield: $C_{2,c} == P_c \implies E_{2,c} == \rho_{c}, \text{  c} \in \{0,1\}$
> ### Total Sale Profits costs: $\mathbb{H}_2 = \sum_{c=1}^{4}(W_{2,c} \cdot F_{2,c})$
> ### Total Purchase costs: $\mathbb{R}_2 = \sum_{c=1}^{2}(Y_{2,c} \cdot C_{2,c})$

### Above Average:
> ### Expected yield below average expected: $E_{3, c} = \rho_{c}(1.2), \forall c$
> ### purchase cost constraint based on yield: $C_{3,c} == P_c \cdot (.9) \implies E_{3,c} == \rho_{c} \cdot (1.2), \text{  c} \in \{0,1\}$
> ### Total Sale Profits costs: $\mathbb{H}_3 = \sum_{c=1}^{4}(W_{3,c} \cdot F_{3,c} \cdot (.9))$
> ### Total Purchase costs: $\mathbb{R}_3 = \sum_{c=1}^{2}(Y_{3,c} \cdot C_{3,c})$

# General Constraints

> ### Minimum feed crop constraints: $Y_{s,c} + E_{s,c} - W_{s,c} \geq M_c, \forall s,c \text{ where c } \in \text{ \{wheat(1), corn(2)\}}$

> ### Sugar Beets Sales constraint: $E_{s,3} - \sum_{c=3}^{4}W_{s,c} \geq 0, \forall s$
> ### Sugar Beet below quota value constraint: $ W_{s,2} \leq 6000, \forall s$



## ############################
# Below Average:
> ### Price average yield
> * $C_{0,c} == P_c*(r_0) \implies E_{0,c} == E_{1,c}*(.8), \text{  c} \in \{0,1\}$
> * $E_{0,c} == E_{1,c}*(.8), \forall c$
# Average:
> ### Price average yield
> * $C_{1,c} == P_c*(r_1) \implies E_{1,c} == E_{1,c}, \text{  c} \in \{0,1\}$
> * $E_{1,c} = X_c \cdot \rho_{s,c}, \forall c$
# Above Average: 
> ### Price average yield
> * $C_{2,c} == P_c*(r_2) \implies E_{2,c} == E_{1,c}*(1.2), \text{  c} \in \{0,1\}$
> * $E_{2,c} == E_{1,c}*(1.2), \forall c$
> * $ \sum_{s}^{|S|}((\sum_{c=2}^{3}W_{s,c}) \leq E_{s,c})$
> * $ \sum_{s}^{|S|}\sum_{c=0}^{1}((W_{s,c}) \leq E_{s,c})$
> * $ W_{s,2} \leq 6000, \forall s$

## <a id=objective-2><span style="color:DarkBlue">Obective:</span></a>
> ## Maximize: $\sum_{s=1}^{3}((\mathbb{H}_s - \mathbb{R}_s) \cdot \mu_s ) - \mathbb{G}$

# <a id=prob2-gurobi><center>Gurobi Implementation and Solution Problem 2</center></a>

In [57]:
try:
    # instantiate model object 
    m = gp.Model("Farmer_problem2")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    C = 3       # number of crop lots 
    S = 3       # number of possible scenerios
    F = len(feedCrops)        # amount of feed crops
    A = 500
    display(data_df)
    crop_yieldRate_scenerio = [.8, 1, 1.2]  # below average, average, above average
    crop_price_scenerio =[1.1, 1, .9]  # below average, average, above average
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    Xc = m.addVars(C, vtype=GRB.CONTINUOUS, name="X", lb=0)    # acreage per crop for planting
    Esc = m.addVars(S, C, vtype=GRB.CONTINUOUS, name="E", lb=0,)  # expected yield for each crop & scenerio
    Ysc = m.addVars(S, F, vtype=GRB.CONTINUOUS, name="Y", lb=0)   # amount to purchase for feed crops
    Csc = m.addVars(S, F, vtype=GRB.CONTINUOUS, name="C", lb=0)   # purchase costs for feed crops & scenerio
    Wsc = m.addVars(S, C+1, vtype=GRB.CONTINUOUS, name="W", lb=0)   # amount to sell per crop
    
    G =   m.addVars(1, vtype=GRB.CONTINUOUS, name="G", lb=0,)     # planting costs
    Hs = m.addVars(S, vtype=GRB.CONTINUOUS, name="H", lb=0) # profits for sales in each scenerio for each crop
    Rs = m.addVars(S, vtype=GRB.CONTINUOUS, name="R", lb=0) # purchase costs for each scenerio for feed crops
    
#     M = m.addVar(1, vtype=GRB.CONTINUOUS, name="M",)           #
#     N = m.addVar(1, vtype=GRB.CONTINUOUS, name="N")
    crops = data_df.columns.to_list()
#     crop_yieldRate_scenerio
#     crop_price_scenerio
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    # set up expected yields for each scenerio
#     for s in range(S):
#         for c in range(C):
#             print(data_df.loc["Yield", crops[c]]*(1+crop_yieldRate_scenerio[s]))
    m.addConstrs(Esc[s, c] == data_df.loc["Yield", crops[c]]*crop_yieldRate_scenerio[s]*Xc[c]  for s in range(S) for c in range(C))

    # set up planting costs (G)
    exprs = 0
    for c in range(C):
        print(data_df.loc["Planting Cost", crops[c]])
        exprs += Xc[c]*data_df.loc["Planting Cost", crops[c]]
    m.addConstr(G[0]==exprs)
    
    # maximum acreage constraint
    tot_acres = 0
    for c in range(C):
        tot_acres += Xc[c]
    m.addConstr(tot_acres <= A)
    # ###################### Feed Crop Constraints
    # set up purchasing cost for different scenerios
    m.addConstrs(Csc[s, c] == data_df.loc["Price", crops[c]]*crop_price_scenerio[s]  for s in range(S) for c in range(F))
    
    # set up minimum feed crop constraints
    m.addConstrs(Ysc[s, c] - Wsc[s, c] + Esc[s, c] >= data_df.loc["min", crops[c]] for s in range(S) for c in range(F))
   
    # set up purchasing costs expressions for each scenerio
    for s in range(S):
        exp = 0
        for c in range(F):
            exp += Ysc[s, c]*data_df.loc["Price", crops[c]]*crop_price_scenerio[s]
        m.addConstr(Rs[s]==exp)
    for s in range(S):
        exp = 0
        for c in range(2):
            exp += Wsc[s, c]*data_df.loc["Selling Cost1", crops[c]]*crop_price_scenerio[s]
        exp += Wsc[s, 2]*data_df.loc["Selling Cost1", crops[2]] + Wsc[s, 3]*data_df.loc["Selling Cost2", crops[2]]
        m.addConstr(Hs[s]==exp)

    below_average_money_flow = (Hs[0] - Rs[0])*(1/3)
    average_money_flow = (Hs[1]  - Rs[1])*(1/3)
    above_average_money_flow = (Hs[2]  - Rs[2])*(1/3)
        
    # ###################### Sugar Beet Constraints
    # set up sugar beet amounts for different levels
    m.addConstrs(Esc[s, 2] - Wsc[s, 2] - Wsc[s, 3] >= 0  for s in range(S))
    m.addConstrs(Esc[s, 2] <= quota  for s in range(S))             # set up lower level of quota sales constraint
    
    # get each scenerios purchasing costs and sales profits
    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    m.setObjective(below_average_money_flow + average_money_flow + above_average_money_flow - G[0], GRB.MAXIMIZE)
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    ## Objective should be for(H(s) - R(s))
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

,Wheat,Corn,Sugar Beets
Yield,2.5,3,20
Planting Cost,150.0,230,260
Selling Cost1,170.0,150,36
Selling Cost2,170.0,150,10
Price,238.0,210,0
min,200.0,240,0


150.0
230
260
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 35 rows, 43 columns and 85 nonzeros
Model fingerprint: 0x2db5ebb0
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [3e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 6e+03]
Presolve removed 28 rows and 28 columns
Presolve time: 0.01s
Presolved: 7 rows, 15 columns, 21 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6213333e+33   6.000000e+30   4.621333e+03      0s
      10    1.0685067e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.068506667e+05
X[0] 170.00000
X[1] 80.00000
X[2] 250.00000
E[0,0] 340.00000
E[0,1] 192.00000
E[0,2] 4000.00000
E[1,0] 425.00000
E[1,1] 240.00000
E[1,2] 5000.00000
E[2,0] 510.00000
E[2,1] 288.00000
E[2,2] 6000.00000
Y[0,0] 0.0000

# <a id=solution2><span style="color:crimson"><center>Solution Discussion: Problem 2</center></a>
> From the results with Gurobi, the optimal lot sizes(X) **for wheat, corn, and sugar beets are 170, 80, and 250 acres, respectively**. This lot-sizing leads to a profit of **$\$$106850.67**. 

>	The **below-average scenario yields 340 tons of wheat, 192 tons of corn, and 4000 tons of sugar beets**. Because there are less than 6KT of sugar beets, it all will be sold at the higher sales price for this scenario. Due to the corn crop generating less than 240 tons of corn for feed, the farmer purchases (Y[0,1]) 48 tons of corn. Thus, this scenario leads to an expected purchasing cost (R[0]) of $\$$11088.00. From sales of excess corn(140T) and beets(4KT)  crops, there is a total profit of (H[0]) $\$$170180.00. 

>	The **average scenario** for this lot-sizing leads to yields of 425, 240, 5000 tons of wheat, corn, and sugar beets, respectively. Therefore, there are 225 tons of wheat and 5K tons of sugar beets to sell for this scenario. There are no excess tons of corn, but there is just enough to meet the feed crop requirements, and so no crops need be purchased by the farmer in the average case. This case leads to no purchases necessary (Y[1,:]) thus no purchasing costs (R[1]) and only sales profits(H[1]) of $\$$218250.00

>	And finally, in the **above-average** scenario, there will be 510, 288, and 6K tons of wheat, corn, and sugar beets, respectively. These yields lead to an excess of 310 tons of wheat, 48 tons of corn, and 6K tons of sugar beets to sell. This lot allotment generates a sales profit (H[2]) of  $\$$269910.00. There is enough of each feed crop to meet the minimum requirements; therefore, there is no need to purchase any additional crops, leading to zero purchasing costs (R[2]). 

In [34]:
crops

['Wheat', 'Corn', 'Sugar Beets']

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 3</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-3)
* [Variables](#Variables-3)
* [Equations and Constraints](#Equations-and-Constraints-3)
* [Objective](#Objective-3)

# <a id=Model-Formulation-1><center> <span style="color:blue"> Model Formulation Problem 2</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-1)
* [Variables](#Variables-1)
* [Equations and Constraints](#Equations-and-Constraints-2)
* [Objective](#Objective-1)

## <a id=Parameters-and-Sets-1><span style="color:DarkBlue">Parameters and Sets:</span></a>

### $\textbf{C}  \quad \quad \quad \text{set of possible crops {wheat(1), corn(2), sugar-beets(3)} } c \in \textbf{C}$ 
### $X_w \quad \quad \quad \text{acres allocated to wheat}$
### $X_c \quad \quad \quad \text{acres allocated to corn}$
### $X_s \quad \quad \quad \text{acres allocated to sugar beets}$
### $S \quad \quad \quad \text{ expected level for yield {below average(1), average(2), below average(3)} where s} \in S$

## <a id=Variables><span style="color:DarkBlue">Variables:</span></a>

### $X_{c} \quad  \text{     amount of acres to devote to crop c } \in C$ 
### $Y_{c,s}   \quad  \text{  tons of crop type c purchased under condition s} \subset \text{{wheat(1), corn(2)}}$
### $W_{c,s}   \quad  \text{T's of crop type c sold under condition s} \subset \text{{wheat(1), corn(2), sugarbeets @ 36 (3), sugarbeets @ 10(4)}}$
### $A   \quad \text{ maximum acres of land available}$
### $M_c\quad \text{ minimum T of crop c required for feed}$
### $M_c\quad \text{ minimum T of crop c required for feed}$


>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

>### <center><span style="font-size:30px;color:red"><b>Acreage Constraint </b></span></center>

# $$\sum_{c=1}^{|C|}(X_c) \leq (A=500)$$

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# $$0 \leq \quad \sum_{c=1}^{|C|}X_{w,c} \quad  \leq H_{w}, \forall w$$

## <a id=Objective><span style="color:green">Objective: </span></a>

# $$\min(T = N + M)$$

# <a id=prob1><center>Problem 1</center></a>
1. [Data Display section](#Data-Display)
2. [Method Definitions](#Method-Definitions)

# <a id=prob1><center>Gurobi Implementation and Solution Problem 1</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("Farmer_problem1")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=prob2><center>Gurobi Implementation and Solution Problem 2</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=prob3><center>Gurobi Implementation and Solution Problem 3</center></a>

In [ ]:
try:
    # instantiate model object 
    m = gp.Model("G_MOD")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################

    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################

    
    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="6")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

# <a id=solution><span style="color:crimson"><center>Solution Discussion</center></a>

> The solution....

In [ ]:
# save the notebook as a pdf
# to_PDF("notebook_title")